In [1788]:
# !pip install docplex
# !pip install qiskit

In [1789]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse.csgraph import laplacian
import copy

In [1790]:
# Reading data from 5 nodes
df1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MLdata/ConcreteData/node1.xlsx')
df2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MLdata/ConcreteData/node2.xlsx')
df3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MLdata/ConcreteData/node3.xlsx')
df4 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MLdata/ConcreteData/node4.xlsx')
df5 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MLdata/ConcreteData/node5.xlsx')

# df1 = pd.read_excel('./node1.xlsx')
# df2 = pd.read_excel('./node2.xlsx')
# df3 = pd.read_excel('./node3.xlsx')
# df4 = pd.read_excel('./node4.xlsx')
# df5 = pd.read_excel('./node5.xlsx')


In [1791]:
df1.shape, df2.shape, df3.shape, df4.shape, df5.shape

((206, 9), (206, 9), (206, 9), (206, 9), (206, 9))

In [1792]:
node1 = np.asmatrix(df1)
node2 = np.asmatrix(df2)
node3 = np.asmatrix(df3)
node4 = np.asmatrix(df4)
node5 = np.asmatrix(df5)

In [1793]:
np.random.shuffle(node1)
np.random.shuffle(node2)
np.random.shuffle(node3)
np.random.shuffle(node4)
np.random.shuffle(node5)

train_node1 = node1
train_node2 = node2
train_node3 = node3
train_node4 = node4
train_node5 = node5


In [1794]:
ry1, ry2, ry3, ry4, ry5 = copy.deepcopy(node1[:,8]), copy.deepcopy(node2[:,8]), copy.deepcopy(node3[:,8]), copy.deepcopy(node4[:,8]), copy.deepcopy(node5[:,8])

In [1795]:
# parameters setting
y_size = 80  #numbers of y_i
lamda = 10^(-4)
eta = 10^(-5)
rho = 10^(-4)
L = 50


In [1796]:
for i in range(y_size, len(train_node1)):
  train_node1[i,8] = 0
  train_node2[i,8] = 0
  train_node3[i,8] = 0
  train_node4[i,8] = 0
  train_node5[i,8] = 0

In [1797]:
H1, H2, H3, H4, H5 = train_node1[:,0:8], train_node2[:,0:8], train_node3[:,0:8], train_node4[:,0:8], train_node5[:,0:8]
y1, y2, y3, y4, y5 = train_node1[:,8], train_node2[:,8], train_node3[:,8], train_node4[:,8], train_node5[:,8]
I = np.identity(8)
C = np.zeros((206, 206))
L1, L2, L3, L4, L5 = laplacian(H1.dot(H1.transpose()), normed=True), laplacian(H2.dot(H2.transpose()), normed=True), laplacian(H3.dot(H3.transpose()), normed=True), laplacian(H4.dot(H4.transpose()), normed=True), laplacian(H5.dot(H5.transpose()), normed=True)

for i in range(0,y_size+1):
  C[i,i]=1

In [1798]:
H1.shape, y1.shape, I.shape, C.shape, L1.shape, ry1.shape

((206, 8), (206, 1), (8, 8), (206, 206), (206, 206), (206, 1))

In [1799]:
temp1 = H1.transpose().dot(C).dot(H1) + eta * H1.transpose().dot(L1).dot(H1) + lamda * I
temp2 = H2.transpose().dot(C).dot(H2) + eta * H2.transpose().dot(L2).dot(H2) + lamda * I
temp3 = H3.transpose().dot(C).dot(H3) + eta * H3.transpose().dot(L3).dot(H3) + lamda * I
temp4 = H4.transpose().dot(C).dot(H4) + eta * H4.transpose().dot(L4).dot(H4) + lamda * I
temp5 = H5.transpose().dot(C).dot(H5) + eta * H5.transpose().dot(L5).dot(H5) + lamda * I

In [1800]:
P1_inv, P2_inv, P3_inv, P4_inv, P5_inv = np.linalg.inv(temp1), np.linalg.inv(temp2), np.linalg.inv(temp3), np.linalg.inv(temp4), np.linalg.inv(temp5)

In [1801]:
z = np.zeros((8,1))
r1, r2, r3, r4, r5 = np.zeros((8,1)), np.zeros((8,1)), np.zeros((8,1)), np.zeros((8,1)), np.zeros((8,1))
for i in range(0,L):
  #update wi
  w1 = P1_inv.dot(H1.transpose().dot(C).dot(y1) - r1 + rho*z)
  w2 = P2_inv.dot(H2.transpose().dot(C).dot(y2) - r2 + rho*z)
  w3 = P3_inv.dot(H3.transpose().dot(C).dot(y3) - r3 + rho*z)
  w4 = P4_inv.dot(H4.transpose().dot(C).dot(y4) - r4 + rho*z)
  w5 = P5_inv.dot(H5.transpose().dot(C).dot(y5) - r5 + rho*z)

  #update z
  z = 5*((w1 + w2 + w3 + w4 + w5)*rho/5 + (r1 + r2 + r3 + r4 + r5)/5)/(lamda + 5*rho)

  #update ri
  r1 = r1 + rho*(w1 - z)
  r2 = r2 + rho*(w2 - z)
  r3 = r3 + rho*(w3 - z)
  r4 = r4 + rho*(w4 - z)
  r5 = r5 + rho*(w5 - z)



In [1802]:
y1_hat = H1.dot(z)
y2_hat = H2.dot(z)
y3_hat = H3.dot(z)
y4_hat = H4.dot(z)
y5_hat = H5.dot(z)

In [1803]:
error1 = np.linalg.norm(y1_hat-ry1, 2)
MSE1 = error1/206

error2 = np.linalg.norm(y2_hat-ry2, 2)
MSE2 = error2/206

error3 = np.linalg.norm(y3_hat-ry3, 2)
MSE3 = error3/206

error4 = np.linalg.norm(y4_hat-ry4, 2)
MSE4 = error4/206

error5 = np.linalg.norm(y5_hat-ry5, 2)
MSE5 = error5/206

In [1804]:
MSE = (MSE1 + MSE2 + MSE3 + MSE4 + MSE5)/5
MSE

1.1834539709407097

In [1805]:
z

matrix([[ 0.00520067],
        [ 0.004272  ],
        [ 0.00496463],
        [ 0.02806336],
        [ 0.02938577],
        [ 0.01693573],
        [ 0.01513822],
        [-0.01398594]])